In [3]:
# M=[0xFFFFFFFF, 0x4BA5A933, 0x7D910E5A, 0x6D26339C, 0x71C7C3E0, 0x7E0FFC00, 0x731D8E64, 0x6B44F5B0, 0x7DC218EC, 0x4DA1B746, 0x42F0FFFF]
M = [4294967295, 1269147955, 2106658394, 1831220124, 1908917216, 2114976768, 1931316836, 1799681456, 2109872364, 1302443846, 1123090431]

# K=11 bit, S=14 symbols
C11_3gpp = zeros(Complex{Float64}, 2048, 14 * 12)
cw_QPSK = zeros(Complex{Float64}, 16)
cw = zeros(Complex{Float64}, 32);


In [17]:
for C in 0:2047 # C is in the input c-sequence, C=[c0 c1 c2 ... c(K-1)]
    # a
    D = mod(C, 2) * M[1] # D is the d-sequence, D=[d0 d1 d2 ... d31]
    for n in 2:11
        Cₙ = mod(floor(C * 2.0^(1 - n)), 2) * M[n]; @show C, Cₙ 
        D = xor(D, Int64(Cₙ) )
    end

    for n in 0:2:31 # This is pi/2-BPSK
        bit0 = 1 - 2 * mod(floor(D / 2^n), 2)
        bit1 = 1 - 2 * mod(floor(D / 2^(n + 1)), 2)
        cw[1 + n] = bit0
        cw[2 + n] = im * bit1  # to use normal BPSK, remove the pi/2 rotation
        cw_QPSK[1 + div(n, 2)] = (bit0 + im * bit1) / sqrt(2)
    end

    idx = 0
    for symb in 0:13
        for re in 0:11
            if symb == 3 || symb == 10
                C11_3gpp[1 + C, 1 + (symb * 12) + (0:11)] .= ones(Complex{Float64}, 12)
            end

            C11_3gpp[1 + C, 1 .+ (symb * 12) + re] = cw[1 + rem(idx, 32)]  # rate-matching of the cw-sequence
            idx .+= 1
        end
    end
end


(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)
(C, Cₙ) = (0, 0.0)


LoadError: MethodError: no method matching copyto!(::Int64, ::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{0}, Tuple{}, typeof(+), Tuple{Int64, Int64}})

[0mClosest candidates are:
[0m  copyto!([91m::AbstractArray[39m, ::Base.Broadcast.Broadcasted{<:Base.Broadcast.AbstractArrayStyle{0}})
[0m[90m   @[39m [90mBase[39m [90m[4mbroadcast.jl:959[24m[39m
[0m  copyto!([91m::AbstractArray[39m, ::Base.Broadcast.Broadcasted)
[0m[90m   @[39m [90mBase[39m [90m[4mbroadcast.jl:956[24m[39m
[0m  copyto!([91m::AbstractArray[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:937[24m[39m


In [1]:


# C is the code sequence to be generated
C = 77
c0_t = []
c0_t_QPSK = []

for symb in 0:13
    # extract OFDM symbols "symb" from coded sequence and do the DFT-precoding (SC-FDMA in the lecture slides)
    if symb == 3
        symb = 4
    end
    if symb == 10
        symb = 11
    end

    c0_f = fft(C11_3gpp[1 + C, 1 + (symb * 12) + (0:11)])
    c0_f_QPSK = fft(C11_3gpp[1 + C, 1 + (symb * 12) + (0:11)])

    c0_t = [c0_t ifft(c0_f, 2048)]
    c0_t_QPSK = [c0_t_QPSK ifft(c0_f_QPSK, 2048)]
end

using Plots

plot(real(c0_t), label="Real", subplot=211)
plot!(imag(c0_t), label="Imaginary", subplot=212)
plot!(size=(800, 600))

plot(real(c0_t), imag(c0_t), seriestype=:scatter, label="QAM", aspect_ratio=:equal, legend=false)
plot!(size=(800, 800))

plot(real(c0_t_QPSK), imag(c0_t_QPSK), seriestype=:scatter, label="QPSK", aspect_ratio=:equal, legend=false)
plot!(size=(800, 800))


LoadError: DomainError with -1:
Cannot raise an integer x to a negative power -1.
Make x or -1 a float by adding a zero decimal (e.g., 2.0^-1 or 2^-1.0 instead of 2^-1)or write 1/x^1, float(x)^-1, x^float(-1) or (x//1)^-1.